In [87]:
# import libraries

import numpy as np
import string
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords

In [88]:
# read from the json file. Has a shitton of files but hey it works
card_df = pd.read_json(path_or_buf='oracle-cards-20231211100140.json', orient='records')
card_df

,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,produced_mana,card_faces,tcgplayer_etched_id,promo_types,loyalty,life_modifier,hand_modifier,attraction_lights,color_indicator,content_warning
0,card,86bf43b1-8d4e-4759-bb2d-0b2e03ba7012,0004ebd0-dfd6-4276-b4a6-de0003e94237,[15862],15870.0,15871.0,3094.0,3081.0,Static Orb,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,7050735c-b232-47a6-a342-01795bfd0d46,0006faf6-7a61-426c-9034-579f2cfcfa83,[370780],49283.0,49284.0,69965.0,262945.0,Sensory Deprivation,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,e718b21b-46d1-4844-985c-52745657b1ac,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,[470580],77122.0,NaN,196536.0,391692.0,Road of Return,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,036ef8c9-72ac-46ce-af07-83b79d736538,000d5588-5a4c-434e-988d-396632ade42c,[83282],22609.0,22610.0,12835.0,12551.0,Storm Crow,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,b125d1e7-5d9b-4997-88b0-71bdfc19c6f2,000e5d65-96c3-498b-bd01-72b1a1991850,[12380],12637.0,12638.0,6412.0,10604.0,Walking Sponge,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30448,card,d6695fa8-881c-407c-91d9-3ac770372d35,fffa9334-3576-4f70-9605-2ad062cdbc69,[430770],64646.0,64647.0,136683.0,298834.0,Without Weakness,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30449,card,4d5edf4f-5695-42fc-9e57-c4faef60fbc3,fffaa634-42a0-4038-b027-24f28754fec7,[571548],101914.0,NaN,276869.0,665173.0,Firesong and Sunspeaker,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30450,card,22a6a5f1-1405-4efb-af3e-e1f58d664e99,fffcb71f-e802-436e-917d-eaa6607ab74f,"[503766, 503767]",87643.0,NaN,230214.0,530502.0,"Toralf, God of Fury // Toralf's Hammer",en,...,NaN,"[{'object': 'card_face', 'name': 'Toralf, God ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30451,card,09f06f55-7918-46c4-80ff-0bf39e091a4a,fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,[430833],64772.0,64773.0,132234.0,298282.0,"Samut, the Tested",en,...,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN


In [89]:
# This has tons of data we don't need for these purposes, so we'll remove... most of it
# 'language', 'layout', 
#card_df = card_df[['oracle_id', 'color_identity', 'colors', 'keywords', 'legalities', 'mana_cost', 'name', 'oracle_text', 'power', 'produced_mana', 'toughness', 'type_line']]
#print(card_df)

# actually im now realizing that I just want to split the dataframes up into certain related parts
# will keep the oracle_id to keep them all connected

text_df = card_df[['oracle_id', 'name', 'type_line', 'keywords', 'oracle_text']]

mana_df = card_df[['color_identity', 'colors', 'mana_cost', 'cmc', 'produced_mana', 'loyalty']]
meta_df = card_df[['legalities', 'layout', 'edhrec_rank', 'penny_rank', 'reserved', 'set_name']]
text_df

,oracle_id,name,type_line,keywords,oracle_text
0,0004ebd0-dfd6-4276-b4a6-de0003e94237,Static Orb,Artifact,[],"As long as Static Orb is untapped, players can..."
1,0006faf6-7a61-426c-9034-579f2cfcfa83,Sensory Deprivation,Enchantment — Aura,[Enchant],Enchant creature\nEnchanted creature gets -3/-0.
2,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,Road of Return,Sorcery,[Entwine],Choose one —\n• Return target permanent card f...
3,000d5588-5a4c-434e-988d-396632ade42c,Storm Crow,Creature — Bird,[Flying],Flying (This creature can't be blocked except ...
4,000e5d65-96c3-498b-bd01-72b1a1991850,Walking Sponge,Creature — Sponge,[],{T}: Target creature loses your choice of flyi...
...,...,...,...,...,...
30448,fffa9334-3576-4f70-9605-2ad062cdbc69,Without Weakness,Instant,[Cycling],Target creature you control gains indestructib...
30449,fffaa634-42a0-4038-b027-24f28754fec7,Firesong and Sunspeaker,Legendary Creature — Minotaur Cleric,[],Red instant and sorcery spells you control hav...
30450,fffcb71f-e802-436e-917d-eaa6607ab74f,"Toralf, God of Fury // Toralf's Hammer",Legendary Creature — God // Legendary Artifact...,"[Trample, Equip]",NaN
30451,fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,"Samut, the Tested",Legendary Planeswalker — Samut,[],+1: Up to one target creature gains double str...


In [109]:
def printNicely(input, wordsPerLine):
    ugh = input.split(' ')
    #print(ugh)
    i = 0
    while (i < len(input)/wordsPerLine):
        print(' '.join(ugh[i:i+wordsPerLine]))
        i += wordsPerLine

In [110]:
# Start creating BoW model
#text_tk_df = text_df
#text_tk_df['oracle_text'] = text_df['oracle_text'].dropna().apply(word_tokenize)
#text_tk_df
# 154 is apparently the average number of words per card
def generateBOW(textCategory='oracle_text', numberOfWords=154):

    # Turn all the text from one category (pandas column) into a string
    textList = list(text_df[textCategory].dropna().values)
    textListFiltered = [currText.replace('\n', ' ').lower() for currText in textList]
    #print(sum([len(line) for line in textListFiltered])/len(textListFiltered))
    text = ' '.join(textListFiltered)
    tokens = word_tokenize(text)

    # now have a list of every word in every card

    # create a set of all possible words
    possibleWords = list(set(tokens))
    totalWords = len(tokens)
    # initialize counts of words to be zero
    # then count all of them up
    # (to be used for probabilities)
    counts = {word : 0 for word in possibleWords}
    for word in tokens:
        counts[word] += 1
    
    # create probability dictionary where the key is a word
    # and the value is (# instances of the word) / (# all words)
    probs = {word : counts[word]/totalWords for word in possibleWords}
    
    generated = np.random.choice(list(probs.keys()), size=numberOfWords, p=list(probs.values()))
    result = ' '.join(generated)
    return result
# Idea: remove the name from each of them and replace it with the newly generated name for the generated card

wahoo = generateBOW('oracle_text', 50)
#test = "the text on this card reads word1 word2 word3 word4 blah blah enchant target creature until its owner pays 3 life (can only be paid as a sorcery) enchanted creature is stupid and silly"
printNicely(wahoo, 5)

( — each power ,
control this step ability .
. by their green horned
graveyard two to { each
target . to : on
upkeep create get attach of
a blocking the pupil with
1 they have damage creature
opponent put is yawning {
you hand this token mana
